In [ ]:
import os
import sys
import typing as t
from PIL import Image

import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
import skimage.color

# add src to path
sys.path.append('../..')
from utils.db_helper import get_image_data
from utils.color_space_operations import all_colorspaces_from_rgb, calculate_difference_image, get_difference_img_gen, float_to_int_img_conversion

In [ ]:
SOURCE_DIR_STYLE = R"F:\master-thesis-databases\classification_db\fake"
SOURCE_DIR_FFHQ = R"F:\master-thesis-databases\classification_db\real"
base_save_path = os.path.join("..", "..", "..", "exp", "Histograms")


In [ ]:
def calculate_avg_histogram(dataset_gen: t.Generator[np.ndarray, None, None], bins=256, hist_range=(0,1), channels=3):
    number_of_images = 0
    np_avg_hist = np.zeros((bins,channels), dtype=np.float64)
    for np_img in dataset_gen:
        for colorspace in range(channels):
            hist, _ = np.histogram(np_img[:,:,colorspace],                                               density=True, bins=bins, range=hist_range)
            np_avg_hist[:,colorspace] += hist
        number_of_images += 1
        if np_avg_hist.max() > 0.95*np.finfo(np_avg_hist.dtype).max:
            raise ValueError("Dataset to large")
            break

    return np_avg_hist / number_of_images 

## Color componenets histograms

In [ ]:
def get_all_colorspaces_gen(dataset_gen):
    return (all_colorspaces_from_rgb(np_img) for np_img in dataset_gen)

In [ ]:
color_componenets = ("R", "G", "B", "H", "S", "V", "Y", "Cb", "Cr")
save_path = os.path.join(base_save_path, "Color_components_avg_histograms_sharey.png")
categories = ['real', 'fake']


In [ ]:
ffhq_gen = get_image_data(SOURCE_DIR_FFHQ, type='float', grayscale=False, max_number_of_images=100)
style_gen = get_image_data(SOURCE_DIR_STYLE, type='float', grayscale=False, max_number_of_images=100)
real_hist = calculate_avg_histogram(get_all_colorspaces_gen(ffhq_gen), channels=9)
fake_hist = calculate_avg_histogram(get_all_colorspaces_gen(style_gen), channels=9)

In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=3, sharey=True, figsize=(15,7))
fig.suptitle("Average histograms of each color component")
plt.subplots_adjust(hspace=0.5)
for i in range(9):
    axs[int(i/3),i%3].plot(np.linspace(0,1,num=256), real_hist[:,i])
    axs[int(i/3),i%3].plot(np.linspace(0,1,num=256), fake_hist[:,i])
    axs[int(i/3),i%3].legend(['real', 'fake'])
    axs[int(i/3),i%3].set_title(color_componenets[i])
#plt.savefig(save_path)
plt.show()   



## Color Componenets for difference images

In [ ]:
color_componenets = ("R", "G", "B", "H", "S", "V", "Y", "Cb", "Cr")
save_path = os.path.join(base_save_path, "Color_components_diff_avg_histograms_sharey.png")
categories = ['real', 'fake']

In [ ]:
ffhq_gen = get_image_data(SOURCE_DIR_FFHQ, type='float', grayscale=False, max_number_of_images=100)
style_gen = get_image_data(SOURCE_DIR_STYLE, type='float', grayscale=False, max_number_of_images=100)
real_hist = calculate_avg_histogram(get_difference_img_gen(ffhq_gen), channels=9, hist_range=(0,255))
fake_hist = calculate_avg_histogram(get_difference_img_gen(style_gen), channels=9, hist_range=(0,255))

In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=3, sharey=True, figsize=(15,7))
fig.suptitle("Average histograms of each color component in difference image")
plt.subplots_adjust(hspace=0.5)
for i in range(9):
    axs[int(i/3),i%3].plot(np.linspace(0,1,num=256), real_hist[:,i])
    axs[int(i/3),i%3].plot(np.linspace(0,1,num=256), fake_hist[:,i])
    axs[int(i/3),i%3].legend(['real', 'fake'])
    axs[int(i/3),i%3].set_title(color_componenets[i])
# plt.savefig(save_path)
plt.show()  